[View in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/wosplus.ipynb)

In [0]:
!pip install openpyxl xlrd unidecode python-levenshtein requests_testadapter > /dev/null

In [0]:
rm -rf WOSplus

In [3]:
! git clone https://github.com/restrepo/WOSplus.git > /dev/null

Cloning into 'WOSplus'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 137 (delta 30), reused 24 (delta 10), pack-reused 83
Receiving objects: 100% (137/137), 114.56 KiB | 19.09 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [0]:
mv WOSplus/wosplus/* .

In [0]:
import wosplus as wp

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
filetxt="drive/Team Drives/Medición Capacidades Vinculación UDEA/06. Cienciometría/1. Descargas/1. Web of Science/1. UDEA/savedrecs.txt"

In [0]:
%%writefile drive.cfg
[FILES]
Sample_WOS.xlsx  = 0BxoOXsn2EUNIMldPUFlwNkdLOTQ
Sample_WOS.txt   = 12CtQ_SI2OHrvj_etKpqriGsGoVvv9zkL
savedrecs01.txt  = 1snzdsa-RLwYIf8MUffauaD2ZjNr1U2Os

Overwriting drive.cfg


In [0]:
WOS=wp.wosplus('drive.cfg')

In [0]:
WOS.load_biblio(filetxt)
WOS.WOS[:2]

,AB,AF,AR,AU,C1,CR,DA,DE,DI,DT,...,SI,SP,PM,OA,RI,MA,SU,PN,CA,Tipo
0,,"Burwick, Richard M.\nVelasquez, Jesus A.\nVale...",,"Burwick, RM\nVelasquez, JA\nValencia, CM\nBern...","[Burwick, Richard M.; Tolosa, Jorge E.] Oregon...",,2018-02-15\n,,,Meeting Abstract\n,...,,Soc Maternal Fetal Med\n,,,,306\n,S\n,,,WOS
1,,"Burwick, Richard M.\nVelasquez, Jesus A.\nVale...",,"Burwick, RM\nVelasquez, JA\nValencia, CM\nSilv...","[Burwick, Richard M.; Rincon, Monica; Tolosa, ...",,2018-02-15\n,,,Meeting Abstract\n,...,,Soc Maternal Fetal Med\n,,,,305\n,S\n,,,WOS


In [0]:
def future_download_file_from_google_drive(id='1snzdsa-RLwYIf8MUffauaD2ZjNr1U2Os',file=None):
    '''
    Download file from Google Drive public id
    If 
      file=None
    returns: 
      * File object for a binary file
        Example: 
          txt=future_download_public_drive_file()
          print(txt[:2])
          #OR
          with open('kk.txt', 'w') as f:
             f.write(txt)
      * String for a txt file
        Example:
          import pandas as pd
          pd.read_excel(  future_download_public_drive_file(id='0BxoOXsn2EUNIMldPUFlwNkdLOTQ')  )[:1]
    
    '''    

In [0]:
def save_response_content(response, destination=None,binary=True):
    CHUNK_SIZE = 32768

    if destination:
        f=open(destination, "wb") #binary file
    else:
        chunks=b''
    for chunk in response.iter_content(CHUNK_SIZE):
        if chunk: # filter out keep-alive new chunks
            if destination:
                f.write(chunk)
            else:
                chunks=chunks+chunk
    if destination:
        f.close()  #writes the file
    else:
        if binary:
            return io.BytesIO(chunks) # returns the file object
        else:
            return io.StringIO(chunks.decode()) # returns the file object

In [0]:
chunks=b''
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                    chunks=chunks+chunk

        try: 
            ior=io.StringIO(chunks.decode())
        except  UnicodeDecodeError:
            ior=io.BytesIO(chunks)

        # returns the file object
        return ior
def future_response_content(file=None,id='1snzdsa-RLwYIf8MUffauaD2ZjNr1U2Os'):
    '''
    Download file from Google Drive public id
    If 
      file=None
    returns: 
      * File object for a binary file
        Example: 
          txt=future_download_public_drive_file()
          print(txt[:2])
          #OR
          with open('kk.txt', 'w') as f:
             f.write(txt)
      * String for a txt file
        Example:
          import pandas as pd
          pd.read_excel(  future_download_public_drive_file(id='0BxoOXsn2EUNIMldPUFlwNkdLOTQ')  )[:1]
    
    '''
    import requests as r
    import io
    CHUNK_SIZE = 32768
    
    response=r.get('https://docs.google.com/uc?export=download&id='+id)

    if file:
        with open(file, 'wb') as f:
            f.write(response.content)
        return
    else:
        

'drive/Team Drives/Medición Capacidades Vinculación UDEA/06. Cienciometría/1. Descargas/1. Web of Science/1. UDEA/savedrecs.txt'

In [6]:
import requests
import io
from configparser import ConfigParser
'''
Based on:
http://stackoverflow.com/a/39225272

To change the global variables cfg and drive_file use

self.ReadConfig('drive.cfg')

for some 'drive.cfg' file with format:

$ cat drive.cfg
[FILES]
Sample_WOS.xlsx = 0BxoOXsn2EUNIMldPUFlwNkdLOTQ
'''
# ====  BEGIN: Global variables =====
ConfigParserFile='drive.cfg'

def load_cfg():
    from configparser import ConfigParser    
    cfg=ConfigParser()
    cfg.optionxform=str

    return cfg

def ReadConfig(CPfile):
    from pathlib import Path
    cfg=load_cfg()
    my_file = Path(CPfile)
    if my_file.is_file():
        tmp=cfg.read(CPfile)
    else:
        tmp=cfg.read_dict({'FILES':
                    {'Sample_WOS.xlsx':'0BxoOXsn2EUNIMldPUFlwNkdLOTQ',
                     'Sample_WOS.txt' :'12CtQ_SI2OHrvj_etKpqriGsGoVvv9zkL'}})
        
    return cfg

cfg=ReadConfig(ConfigParserFile)
drive_file=cfg['FILES']

# ====  END: Global variables =====

def load_drive_files_keys(CPfile=ConfigParserFile):
    cfg=load_cfg()
    tmp=cfg.read(CPfile)
    return cfg['FILES']

    
def pandas_from_google_drive_csv(id,gss_sheet=0,gss_query=None):
    '''
    Read Google spread sheet by id.
    Options:
       gss_sheet=N : if in old format select the N-sheet
       gss_query=SQL_command: Filter with some SQL command
       example 
         SQL_command: 'select B,D,E,F,I where (H contains 'GFIF') order by D desc'
    '''
    import pandas as pd
    if not gss_query:
        url='https://docs.google.com/spreadsheets/d/%s/gviz/tq?tqx=out:csv&gid=%s' %(id,gss_sheet)
    else:
        url='https://docs.google.com/spreadsheets/d/%s/gviz/tq?tqx=out:csv&gid=%s&tq=%s' %(id,gss_sheet,gss_query)
    r=requests.get(url)
    if r.status_code==200:
        csv_file=io.StringIO(r.text) # or directly  with: urllib.request.urlopen(url)
        return pd.read_csv( csv_file,keep_default_na=False)

def download_file_from_google_drive(id,destination=None,binary=True):
    '''
    Download file from google drive as binary (default) or txt file.
    If not destination the file object is returned
    Example: Let id="XXX" a txt file:
    1) fb=download_file_from_google_drive("XXX") ; fb.decode() #to convert to text file
    2) ft=download_file_from_google_drive("XXX",binary=False) # txt file
    3) fb=download_file_from_google_drive("XXX",'output_file') # always binay
    '''
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    
    return save_response_content(response, destination=destination,binary=binary)


def download_file_from_local_drive(localfile,destination=None,binary=True):
    '''
    Download file from google drive as binary (default) or txt file.
    If not destination the file object is returned
    Example: Let id="XXX" a txt file:
    1) fb=download_file_from_google_drive("XXX") ; fb.decode() #to convert to text file
    2) ft=download_file_from_google_drive("XXX",binary=False) # txt file
    3) fb=download_file_from_google_drive("XXX",'output_file') # always binay
    '''
    URL = "file://"

    session = requests.Session()
    
    session.mount('file://', LocalFileAdapter())
    
    r=requests_session.get('file://./Sample_WOS.txt')

    response = session.get(URL+'./'+localfile)
    token = get_confirm_token(response)
    
    return save_response_content(response, destination=destination,binary=binary)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination=None,binary=True):
    CHUNK_SIZE = 32768

    if destination:
        f=open(destination, "wb") #binary file
    else:
        chunks=b''
    for chunk in response.iter_content(CHUNK_SIZE):
        if chunk: # filter out keep-alive new chunks
            if destination:
                f.write(chunk)
            else:
                chunks=chunks+chunk
    if destination:
        f.close()  #writes the file
    else:
        if binary:
            return io.BytesIO(chunks) # returns the file object
        else:
            return io.StringIO(chunks.decode()) # returns the file object
        
def read_drive_excel(file_name,**kwargs):
    '''Read excel or csv file from google drive
     Requieres a drive_file dictionary with the keys for the google drive
     file names.
     If the file_name is not found in the drive_file dictionary it is read locally.
     If the file_name have an extension .csv, tray to read the google spreadsheet 
     directly: check pandas_from_google_drive_csv for passed options
     WARNING: ONLY OLD Google Spread Sheet allows to load sheet different from 0
     '''
    import pandas as pd
    import re
    if re.search('\.csv$',file_name):
        if drive_file.get(file_name):
            return pandas_from_google_drive_csv(drive_file.get(file_name),**kwargs)
        else:
            return pd.read_csv(file_name,**kwargs)

    if drive_file.get(file_name):    
        return pd.read_excel( download_file_from_google_drive(
            drive_file.get(file_name) ) ,**kwargs)  # ,{} is an accepted option   
    else:
        return pd.read_excel(file_name,**kwargs)
    

def query_drive_csv(
    gss_url="https://spreadsheets.google.com",
    gss_format="csv",
    gss_key="0AuLa_xuSIEvxdERYSGVQWDBTX1NCN19QMXVpb0lhWXc",
    gss_sheet=0,
    gss_query="select B,D,E,F,I where (H contains 'GFIF') order by D desc",
    gss_keep_default_na=False
    ):
    import pandas as pd
    issn_url="%s/tq?tqx=out:%s&tq=%s&key=%s&gid=%s" %(gss_url,\
                                           gss_format,\
                                           gss_query,\
                                           gss_key,\
                                           str(gss_sheet))

    return pd.read_csv( issn_url.replace(' ','+') )

import requests
import io
from configparser import ConfigParser
'''
Based on:
http://stackoverflow.com/a/39225272

To change the global variables cfg and drive_file use

self.ReadConfig('drive.cfg')

for some 'drive.cfg' file with format:

$ cat drive.cfg
[FILES]
Sample_WOS.xlsx = 0BxoOXsn2EUNIMldPUFlwNkdLOTQ
'''
# ====  BEGIN: Global variables =====
ConfigParserFile='drive.cfg'

def load_cfg():
    from configparser import ConfigParser    
    cfg=ConfigParser()
    cfg.optionxform=str

    return cfg

def ReadConfig(CPfile):
    from pathlib import Path
    cfg=load_cfg()
    my_file = Path(CPfile)
    if my_file.is_file():
        tmp=cfg.read(CPfile)
    else:
        tmp=cfg.read_dict({'FILES':
                    {'Sample_WOS.xlsx':'0BxoOXsn2EUNIMldPUFlwNkdLOTQ',
                     'Sample_WOS.txt' :'12CtQ_SI2OHrvj_etKpqriGsGoVvv9zkL'}})
        
    return cfg

cfg=ReadConfig(ConfigParserFile)
drive_file=cfg['FILES']

# ====  END: Global variables =====

def load_drive_files_k